In [24]:
!pip install openai

In [53]:
def read_secrets(config_file):
    with open(config_file, "r") as file:
        data = file.read()
        data = json.loads(data)
        file.close()
        for item in data:
            os.environ[item] = data[item]

read_secrets("secrets.json")

In [59]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
#from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
import requests
import json
import os

#### Nutritionix Request Test

In [60]:
url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
headers = {
    'Content-Type': 'application/json',
    'x-app-id': os.environ['APPLICATION_ID'],
    'x-app-key':os.environ['APPLICATION_KEY'],
  #  'x-remote-user-id': 0
  }
body = {
    "query": "egg omelette with three eggs"
  }
response = requests.post(url=url, headers=headers, json=body)


In [61]:
response.json()

{'foods': [{'food_name': 'egg omelette',
   'brand_name': None,
   'serving_qty': 1,
   'serving_unit': '3-egg omelet',
   'serving_weight_grams': 178.38,
   'nf_calories': 323.47,
   'nf_total_fat': 25.32,
   'nf_saturated_fat': 5.86,
   'nf_cholesterol': 624.96,
   'nf_sodium': 533.35,
   'nf_total_carbohydrate': 1.39,
   'nf_dietary_fiber': 0.07,
   'nf_sugars': 0.62,
   'nf_protein': 21.13,
   'nf_potassium': 235.72,
   'nf_p': 333.09,
   'full_nutrients': [{'attr_id': 203, 'value': 21.1307},
    {'attr_id': 204, 'value': 25.3197},
    {'attr_id': 205, 'value': 1.3935},
    {'attr_id': 207, 'value': 2.5526},
    {'attr_id': 208, 'value': 323.47},
    {'attr_id': 210, 'value': 0.0001},
    {'attr_id': 211, 'value': 0.6223},
    {'attr_id': 212, 'value': 0.0007},
    {'attr_id': 213, 'value': 0},
    {'attr_id': 214, 'value': 0},
    {'attr_id': 221, 'value': 0},
    {'attr_id': 255, 'value': 127.9693},
    {'attr_id': 262, 'value': 0},
    {'attr_id': 263, 'value': 0},
    {'attr_id

#### LLM Open AI

In [81]:
from getpass import getpass

OPENAI_API_KEY = getpass()

In [82]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [54]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent

llm = ChatOpenAI()

### Agent Building

In [129]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

# class FoodSearchInput(BaseModel):
#     query: str = Field(description="description of a single food")

@tool
def FoodLookup(query: str) -> dict:
    "Function to lookup food nutrion facts"
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': os.environ['APPLICATION_ID'],
        'x-app-key':os.environ['APPLICATION_KEY'],
    #  'x-remote-user-id': 0
    }
    body = {
        "query": query
    }
    response = requests.post(url=url, headers=headers, json=body)
    return response.json()

# foodLookupTool = StructuredTool.from_function(
#     func=FoodLookup,
#     name="Food Lookup Tool",
#     description="lookup the nutrition information of a specfic food",
#     args_schema=FoodSearchInput,
#     # coroutine= ... <- you can specify an async method if desired as well
# )

In [130]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import MessagesPlaceholder

system_prompt = (
    "You are a helpful AI bot."
    "That is skilled at calculating calories and nutrion facts based on text input."
    "You have a tool that is able to find caloric information based on a string query."
    "Your job is to return the total calories and grams of protein consumed"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Today I ate {food_query}. What is the total calories and protein in it?"),
    MessagesPlaceholder("agent_scratchpad")
    
])

tools = [FoodLookup]

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [131]:
agent_executor.invoke(input={'food_query': "One cup, Chobani Greek Yogurt, 1/2 cup Granola"})



> Entering new AgentExecutor chain...

Invoking: `FoodLookup` with `{'query': 'One cup Chobani Greek Yogurt'}`


{'foods': [{'food_name': 'greek yogurt', 'brand_name': None, 'serving_qty': 1, 'serving_unit': 'cup', 'serving_weight_grams': 226.67, 'nf_calories': 133.74, 'nf_total_fat': 0.88, 'nf_saturated_fat': 0.27, 'nf_cholesterol': 11.33, 'nf_sodium': 81.6, 'nf_total_carbohydrate': 8.16, 'nf_dietary_fiber': 0, 'nf_sugars': 7.34, 'nf_protein': 23.1, 'nf_potassium': 319.6, 'nf_p': 306, 'full_nutrients': [{'attr_id': 203, 'value': 23.0977}, {'attr_id': 204, 'value': 0.884}, {'attr_id': 205, 'value': 8.1601}, {'attr_id': 207, 'value': 1.632}, {'attr_id': 208, 'value': 133.7353}, {'attr_id': 210, 'value': 0}, {'attr_id': 211, 'value': 0}, {'attr_id': 212, 'value': 0}, {'attr_id': 213, 'value': 5.7574}, {'attr_id': 214, 'value': 0}, {'attr_id': 221, 'value': 0}, {'attr_id': 255, 'value': 192.8962}, {'attr_id': 262, 'value': 0}, {'attr_id': 263, 'value': 0}, {'attr_id': 268, 'value': 557.

{'food_query': 'One cup, Chobani Greek Yogurt, 1/2 cup Granola',
 'output': 'The total calories and protein in the One cup of Chobani Greek Yogurt and 1/2 cup of Granola are as follows:\n\n- Chobani Greek Yogurt (One cup):\n  - Calories: 133.74\n  - Protein: 23.1 grams\n\n- Granola (1/2 cup):\n  - Calories: 298.29\n  - Protein: 8.34 grams\n\nTotal Calories: 133.74 + 298.29 = 432.03 calories\nTotal Protein: 23.1 + 8.34 = 31.44 grams\n\nTherefore, the total calories consumed are 432.03 calories and the total protein consumed is 31.44 grams.'}

### Response Testing

In [230]:
import requests
import json

url = "http://127.0.0.1:8001/aifitnessapp/query"
data = {
    "query": "Sourdough bread 2 Slices with 2 tablespoons butter"
}

response = requests.post(url=url, json=data)

In [231]:
response.json()

{'query': 'Sourdough bread 2 Slices with 2 tablespoons butter',
 'foodList': ['Sourdough bread', 'butter'],
 'calorieList': [174, 102],
 'proteinList': [7, 0]}

### Langchain Testing and Parsing Writer

In [160]:
from typing import List, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langchain_core.outputs import ChatGeneration, Generation

from langchain.agents.agent import MultiActionAgentOutputParser
import json
from json import JSONDecodeError
from typing import List, Union

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ToolCall,
)
from langchain_core.outputs import ChatGeneration, Generation

from langchain.agents.agent import MultiActionAgentOutputParser
from langchain.agents.output_parsers.tools import (
    ToolAgentAction,
    parse_ai_message_to_tool_action,
)

# class ToolAgentAction(AgentActionMessageLog):
#     tool_call_id: str
#     """Tool call that this message is responding to."""


# def parse_ai_message_to_tool_action(
#     message: BaseMessage,
# ) -> Union[List[AgentAction], AgentFinish]:
#     """Parse an AI message potentially containing tool_calls."""
#     if not isinstance(message, AIMessage):
#         raise TypeError(f"Expected an AI message got {type(message)}")

#     actions: List = []
#     if message.tool_calls:
#         tool_calls = message.tool_calls
#     else:
#         if not message.additional_kwargs.get("tool_calls"):
#             return AgentFinish(
#                 return_values={"output": message.content}, log=str(message.content)
#             )
#         # Best-effort parsing
#         tool_calls = []
#         for tool_call in message.additional_kwargs["tool_calls"]:
#             function = tool_call["function"]
#             function_name = function["name"]
#             try:
#                 args = json.loads(function["arguments"] or "{}")
#                 tool_calls.append(
#                     ToolCall(name=function_name, args=args, id=tool_call["id"])
#                 )
#             except JSONDecodeError:
#                 raise OutputParserException(
#                     f"Could not parse tool input: {function} because "
#                     f"the `arguments` is not valid JSON."
#                 )
#     for tool_call in tool_calls:
#         # HACK HACK HACK:
#         # The code that encodes tool input into Open AI uses a special variable
#         # name called `__arg1` to handle old style tools that do not expose a
#         # schema and expect a single string argument as an input.
#         # We unpack the argument here if it exists.
#         # Open AI does not support passing in a JSON array as an argument.
#         function_name = tool_call["name"]
#         _tool_input = tool_call["args"]
#         if "__arg1" in _tool_input:
#             tool_input = _tool_input["__arg1"]
#         else:
#             tool_input = _tool_input

#         content_msg = f"responded: {message.content}\n" if message.content else "\n"
#         log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
#         actions.append(
#             ToolAgentAction(
#                 tool=function_name,
#                 tool_input=tool_input,
#                 log=log,
#                 message_log=[message],
#                 tool_call_id=tool_call["id"],
#             )
#         )
#     return actions

OpenAIToolAgentAction = ToolAgentAction
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


def parse_ai_message_to_openai_tool_action(
    message: BaseMessage,
) -> Union[List[AgentAction], AgentFinish]:
    """Parse an AI message potentially containing tool_calls."""
    tool_actions = parse_ai_message_to_tool_action(message)
    print("TOOL ACTIONS: ", tool_actions)
    if isinstance(tool_actions, AgentFinish):
        return tool_actions
    final_actions: List[AgentAction] = []
    for action in tool_actions:
        if isinstance(action, ToolAgentAction):
            final_actions.append(
                OpenAIToolAgentAction(
                    tool=action.tool,
                    tool_input=action.tool_input,
                    log=action.log,
                    message_log=action.message_log,
                    tool_call_id=action.tool_call_id,
                )
            )
        else:
            final_actions.append(action)
    return final_actions


class OpenAIToolsAgentOutputParser(MultiActionAgentOutputParser):
    """Parses a message into agent actions/finish.

    Is meant to be used with OpenAI models, as it relies on the specific
    tool_calls parameter from OpenAI to convey what tools to use.

    If a tool_calls parameter is passed, then that is used to get
    the tool names and tool inputs.

    If one is not passed, then the AIMessage is assumed to be the final output.
    """

    @property
    def _type(self) -> str:
        return "openai-tools-agent-output-parser"

    def parse_result(
        self, result: List[Generation], *, partial: bool = False
    ) -> Union[List[AgentAction], AgentFinish]:
        if not isinstance(result[0], ChatGeneration):
            raise ValueError("This output parser only works on ChatGeneration output")
        message = result[0].message
        print("MESSAGE: ", message)
        return parse_ai_message_to_openai_tool_action(message)

    def parse(self, text: str) -> Union[List[AgentAction], AgentFinish]:
        raise ValueError("Can only parse messages")

In [161]:
from typing import Sequence, List

from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool
from langchain_core.utils.function_calling import convert_to_openai_tool

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
#from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from pydantic.v1 import BaseModel
from langchain_core.agents import AgentActionMessageLog, AgentFinish
from openai_tools import OpenAIToolsAgentOutputParserCustom

def create_openai_tools_agent_custom_output(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | JsonOutputFunctionsParser()
    )
    return agent

In [162]:
from pydantic.v1 import BaseModel, Field
from langchain.tools import StructuredTool
import requests
import os

class FoodLookupToolInput(BaseModel):
    query: str = Field(description="The food you wish to look up")

def FoodLookup(query: str):
    "Function to lookup food nutrion facts"
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': os.environ['APPLICATION_ID'],
        'x-app-key':os.environ['APPLICATION_KEY'],
    #  'x-remote-user-id': 0
    }
    body = {
        "query": query
    }
    response = requests.post(url=url, headers=headers, json=body)
    return response.json()

food_lookup_tool = StructuredTool.from_function(
        func=FoodLookup,
        name="food_lookup_tool",
        description="Food Lookup Tool",
        args_schema=FoodLookupToolInput,
        
    )

class Response(BaseModel):
    """Final response to the question being asked"""

    answer: str = Field(description="The final answer to respond to the user")
    sources: List[int] = Field(
        description="List of page chunks that contain answer to the question. Only include a page chunk if it contains relevant information"
    )

def response(answer: str, sources: List[int]):
    return {
        "answer": answer,
        "source": sources
    }

response = StructuredTool.from_function(
        func=response,
        name="Response",
        description="Final response to the question being asked",
        args_schema=Response,
        
    )

In [163]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

tools = [food_lookup_tool]

In [164]:
agent = create_openai_tools_agent_custom_output(llm, tools, prompt)
agent_executor = AgentExecutor(tools=[food_lookup_tool], agent=agent, verbose=True)


In [165]:
response = agent_executor.invoke(
    {"input": "How many calories are in a banana?"},
    return_only_outputs=True,
)



> Entering new AgentExecutor chain...


TypeError: 'NoneType' object is not iterable

In [223]:
from typing import List

from langchain_core.pydantic_v1 import BaseModel as BaseModelResponse
from langchain_core.pydantic_v1 import Field as FieldResponse

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# class Response(BaseModel):
#     """Final response to the question being asked"""

#     answer: str = Field(description="The final answer to respond to the user")
#     total_calories: str = Field(description="The total number of calories in all of the foods combined")
#     total_protein: str = Field(description="The total amount of protein in all of the foods combined")

class Response(BaseModelResponse):
    foodList: List[str] = FieldResponse(description="a list of all the foods that the user inputted")
    calorieList: List[int] = FieldResponse(description="A list of the number of calories in each food the user submitted")
    proteinList: List[int] = FieldResponse(description="A list of the protein that was in each food the user submitted")
    
def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

system_prompt = (
    "You are a helpful AI bot."
    "That is skilled at calculating calories and nutrion facts based on text input."
    "You have a tool that is able to find caloric information based on a string query."
    "Your job is to return the total calories and grams of protein consumed given the following food"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm = ChatOpenAI(temperature=0)
llm_with_tools = llm.bind_functions([food_lookup_tool, Response])
agent = (
    {
        "query": lambda x: x["query"],
        # Format agent scratchpad from intermediate steps
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | parse
)

agent_executor = AgentExecutor(tools=[food_lookup_tool], agent=agent, verbose=True)




In [224]:
response = agent_executor.invoke({"query": "How many calories are in a banana and a cup of milk"})



> Entering new AgentExecutor chain...
{'foods': [{'food_name': 'banana', 'brand_name': None, 'serving_qty': 1, 'serving_unit': 'medium (7" to 7-7/8" long)', 'serving_weight_grams': 118, 'nf_calories': 105.02, 'nf_total_fat': 0.39, 'nf_saturated_fat': 0.13, 'nf_cholesterol': 0, 'nf_sodium': 1.18, 'nf_total_carbohydrate': 26.95, 'nf_dietary_fiber': 3.07, 'nf_sugars': 14.43, 'nf_protein': 1.29, 'nf_potassium': 422.44, 'nf_p': 25.96, 'full_nutrients': [{'attr_id': 203, 'value': 1.2862}, {'attr_id': 204, 'value': 0.3894}, {'attr_id': 205, 'value': 26.9512}, {'attr_id': 207, 'value': 0.9676}, {'attr_id': 208, 'value': 105.02}, {'attr_id': 209, 'value': 6.3484}, {'attr_id': 210, 'value': 2.8202}, {'attr_id': 211, 'value': 5.8764}, {'attr_id': 212, 'value': 5.723}, {'attr_id': 213, 'value': 0}, {'attr_id': 214, 'value': 0.0118}, {'attr_id': 221, 'value': 0}, {'attr_id': 255, 'value': 88.3938}, {'attr_id': 262, 'value': 0}, {'attr_id': 263, 'value': 0}, {'attr_id': 268, 'value': 437.78}, {'at

In [216]:
response

{'query': 'How many calories are in a banana and a cup of milk',
 'foodList': ['banana', 'milk'],
 'calorieList': [105, 122],
 'proteinList': [1, 8]}